In [3]:
# Importar librerias y lenguajes
import spacy 
from spacy.tokens import Token, Span
# nlp = spacy.blank("es") # crea un pipeline vacio para procesar el espanol.
nlp = spacy.load("es_core_news_md") # Descarga el paquete del lenguaje espanol preentrenado para poder trabajar con las capacidades centrales.

from spacy.matcher import Matcher # Importo el Matcher.
matcher = Matcher(nlp.vocab) # Inicializo el matcher.

from spacy.tokens import Doc, Span

In [4]:
# Crear el texto, en un objeto llamado "doc".
doc = nlp("Arroz con leche comer, me quiero casar"
        " Con uno señora 5 de San Nicolás"
        " Que sepa cocer, que sepa bordar"
        " Que sepa el abrir la puerta para ir a jugar"
        " Yo soy 20 la viudita del barrio del rey"
        " Me quiero casar y no sé con quién"
        " Con esta sí, con esta no"
        " Con esta 40 señorita me caso yo."
        " Arroz con leche, me quiero casar"
        " Con una señorita de san nicolás"
        " Que sepa  la cocer, que sepa bordar"
        " Que sepa abrir la puerta para ir a jugar"
        " Yo soy la viudita del barrio del rey"
        " Me quiero casar y no sé con quién"
        " Con esta sí, con esta no"
        " Con esta señorita me caso yo")

# Iterar sobre los tokens en el doc. Spacy transforma cada palabra, signo, puntuacion del doc en un token.
for token in doc:
    print(token.text)

Arroz
con
leche
comer
,
me
quiero
casar
Con
uno
señora
5
de
San
Nicolás
Que
sepa
cocer
,
que
sepa
bordar
Que
sepa
el
abrir
la
puerta
para
ir
a
jugar
Yo
soy
20
la
viudita
del
barrio
del
rey
Me
quiero
casar
y
no
sé
con
quién
Con
esta
sí
,
con
esta
no
Con
esta
40
señorita
me
caso
yo
.
Arroz
con
leche
,
me
quiero
casar
Con
una
señorita
de
san
nicolás
Que
sepa
 
la
cocer
,
que
sepa
bordar
Que
sepa
abrir
la
puerta
para
ir
a
jugar
Yo
soy
la
viudita
del
barrio
del
rey
Me
quiero
casar
y
no
sé
con
quién
Con
esta
sí
,
con
esta
no
Con
esta
señorita
me
caso
yo


In [5]:
# El objeto doc tiene su propio indice para el acceso directo.
token = doc[5]
print(token.text) # la funcion .text devuelve el texto.
print("---------------------")
# Un objeto SPAN es un slice del doc, compuesto por 1 o mas tokens. Es un view, no contiene los datos en si.
span = doc[4:13]
print(span)
print("---------------------")
print("Índice:   ", [token.i for token in doc[0:10]])
print("---------------------")
# Reconocimiento de numeros, puntuaciones y palabras alfabeticas (llamados tambien atributos lexicos, no dependen del contexto):
print("is_alpha:", [token.is_alpha for token in doc[0:10]])
print("is_punct:", [token.is_punct for token in doc[0:10]])
print("like_num:", [token.like_num for token in doc[0:10]])

me
---------------------
, me quiero casar Con uno señora 5 de
---------------------
Índice:    [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
---------------------
is_alpha: [True, True, True, True, False, True, True, True, True, True]
is_punct: [False, False, False, False, True, False, False, False, False, False]
like_num: [False, False, False, False, False, False, False, False, False, True]


In [6]:
# Itera sobre los tokens en el doc
for token in doc:
    # Revisa si el token parece un número
    if token.like_num:
        # Obtén el próximo token en el documento
        next_token = doc[token.i + 1]
        # Revisa si el texto del siguiente token es igual a '%'
        if next_token.text == "señorita":
            print("Token encontrado:", (token.text))

Token encontrado: 40


In [7]:
# Componentes del pipeline entrenados.

# Parser (Etiquetas gramaticales)
for token in doc[0:10]:
    # Imprime en pantalla el texto y la etiqueta gramatical predicha
    print(token.text, token.pos_) # En spaCy, los atributos que devuelven un string normalmente terminan con un guion bajo (_). 
                                  # Mientras que atributos sin un guion bajo devuelven un valor ID de tipo entero.

Arroz PROPN
con ADP
leche NOUN
comer VERB
, PUNCT
me PRON
quiero VERB
casar VERB
Con ADP
uno PRON


In [8]:
# Tagger (Relaciones entre las palabras)
for token in doc[0:10]:
    print(token.text, token.dep_, token.head.text) # El atributo .dep_ devuelve la etiqueta de la dependencia sintáctica predicha.
                                                               # El atributo .head devuelve el token de la cabeza de la dependencia sintáctica.

Arroz nsubj comer
con case leche
leche obl comer
comer advcl quiero
, punct comer
me iobj quiero
quiero ROOT quiero
casar xcomp quiero
Con case señora
uno det señora


In [9]:
#Entity recognizer (entidades nombradas: objetos de la vida real que tienen un nombre asignado)

for ent in doc.ents:
    # Imprime en pantalla el texto y la etiqueta de la entidad
    print(ent.text, ent.label_) # La propiedad doc.ents te permite acceder a las entidades nombradas predichas por el modelo de reconocimiento de entidades.
                                # La propiedad doc.ents devuelve un iterador de objetos Span, 
                                # así que podemos imprimir en pantalla el texto y la etiqueta de la entidad usando el atributo .label_.

spacy.explain("MISC") # Explica las definiciones de las etiquetas.
# Mal resultado.

Con uno señora 5 de San Nicolás Que MISC
Me PER
Con esta 40 señorita me caso yo. MISC
Arroz PER
Con una señorita de san nicolás Que sepa  la cocer MISC
Me PER
Con esta señorita me caso yo MISC


'Miscellaneous entities, e.g. events, nationalities, products or works of art'

In [10]:
# Matcher. permite escribir reglas para encontrar palabras y frases en el texto.
# Funciona con objetos Doc y Span, no solo con strings.
# Identifica la función de una palabra teniendo en cuenta su contexto.
# Los patrones del Matcher son listas de diccionarios. Cada diccionario describe un Token.

# Busca por textos exactos de tokens
#patternExacto = [{"TEXT": "Arroz"}, {"TEXT": "con"}] # Crea el patron
#matcher.add("PATRON_TEXTO",[patternExacto]) # Añadir el patron

#Busca por atributos léxicos
#patternMayus = [{"LOWER": "san"}, {"LOWER": "nicolás"}]
#matcher.add("PATRON_MAYUS",[patternMayus])               # NO SABEMOS PORQUE LO DEVUELVE 2 VECES (creemos que porque encuntra
                                                         # uno en mayus y devuelve como si fuese uno en mayus y otro en minus)

# Busca por cualquier atributo del token
#patternRaiz = [{"LEMMA": "querer"}, {"POS": "VERB"}]
#matcher.add("PATRON_RAIZ",[patternRaiz])

#patternDigito = [{"IS_DIGIT": True}]
#matcher.add("PATRON_DIGITO",[patternDigito])

#patternSigno = [{"IS_PUNCT": True}]
#matcher.add("PATRON_SIGNO",[patternSigno])

# Los operadores y cuantificadores te permiten definir con qué frecuencia un token debe ser encontrado. 
# Pueden ser añadidos con el key "OP".

# "OP" puede tener uno de cuatro valores:
# Un "!" niega el token, así que es buscado 0 veces.
# Un "?" hace que el token sea opcional y es buscado 0 o 1 veces.
# Un "+" busca el token 1 o más veces.
# Finalmente, un "*" busca 0 o más veces.

patternOpcional = [{"TEXT": "sepa"},{"POS": "DET", "OP": "+"},{"POS": "VERB"}]  # opcional: encuentra 0 o 1 ocurrencias
matcher.add("PATRON_OPCIONAL",[patternOpcional])


matches = matcher(doc)   # Devuelve una lista de tuples.
                         # Cada Tuple consiste de 3 valores: ID del resultado, indice de inicio y indice de final del span resultante.

for match_id, start,end in matches: 
    matched_span = doc[start:end]
    print(matched_span.text)

sepa el abrir


In [11]:
# spaCy guarda todos los datos en un vocabulario, el Vocab.
# Este incluye palabras, pero también los esquemas de etiquetas y entidades.
# Para usar menos memoria, todos los strings son codificados a hash IDs. 
# Si una palabra ocurre múltiples veces, no tenemos que guardarla cada vez.
# En cambio, spaCy usa una función hash para generar un ID y guarda el string una vez en el string store. 
# El string store está disponible como nlp.vocab.strings

# Es una lookup table que funciona en ambas direcciones. Puedes buscar un string y obtener su hash, 
# así como buscar un hash para obtener su valor string. Internamente spaCy solo se comunica en hash IDs.

# Los hash IDs no se pueden revertir. Si una palabra no está en el vocabulario no hay forma de obtener su string. 
# Es por esto que siempre tenemos que pasar el vocabulario compartido.

nlp.vocab.strings.add("café") # Vocab: guarda los datos compartidos a través de múltiples documentos.
cafe_hash = nlp.vocab.strings["café"] # Guarda el hash del string "café".
cafe_string = nlp.vocab.strings[cafe_hash] # Guarda el string "café" dado su hash.
doc1 = nlp("Ines toma café")
print("hash value:", doc1.vocab.strings["café"])
print(cafe_hash)
print(cafe_string)


hash value: 32833993555699147
32833993555699147
café


In [12]:
# Los lexemas son entradas en el vocabulario independientes del contexto.
# Puedes obtener un lexema buscando un string o un hash ID en el vocabulario.
# Los lexemas exponen atributos, al igual que los tokens.
# Ellos contienen información sobre una palabra independiente del contexto, 
# como el texto o si la palabra está compuesta por caracteres alfanuméricos.
# Los lexemas no tienen etiquetas gramaticales, dependencias o etiquetas de entidades, ya que esos dependen del contexto.

lexeme = nlp.vocab["Arroz"]

# Imprime en pantalla los atributos léxicos
print(lexeme.text, lexeme.orth, lexeme.is_alpha)


Arroz 4826447396574439791 True


In [13]:

# Busco un sustantivo seguido de un verbo.
for token in doc:
    if token.pos_ == "NOUN":  # Revisa si el token actual es un nombre propio
        if doc[token.i + 1].pos_ == "VERB": # Revisa si el siguiente token es un verbo
            print("Encontré un nombre propio antes de un verbo:", token.text)

Encontré un nombre propio antes de un verbo: leche


In [14]:
#### COMENTO TODO PORQUE PIERDO LAS RELACIONES DE LAS PALABRAS.


# Crear un doc manualmente.
# Las palabras y espacios que usaremos para crear el doc
#words = ["¡", "Hola", "Mundo", "!"]
#spaces = [False, True, False, False] # Los espacios son una lista de valores booleanos que indican si una palabra está seguida por un espacio.

# Crea un doc manualmente
#doc = Doc(nlp.vocab, words=words, spaces=spaces) # La clase Doc recibe tres argumentos: el vocabulario compartido, las palabras y los espacios.
#print(doc)

#span = Span(doc, 1, 3) # El Span recibe al menos tres argumentos: el doc al que se refiere, el índice de inicio y el índice del final del span.
#print(span)

# Crea un span con un label
#span_with_label = Span(doc, 1, 3, label="SALUDO")

# Añade el span a los doc.ents
#doc.ents = [span_with_label]

# Si la aplicación tiene que generar strings asegúrate de convertir el doc lo más tarde posible. 
# Si lo haces demasiado temprano perderás las relaciones entre los tokens.


In [15]:
# Comparar similitud entre un span con un documento
span = nlp("Me gusta la pizza con queso")[3:6]
doc = nlp("McDonalds vende hamburguesas con queso")

print(span.similarity(doc))

# La similitud se determina usando word vectors, que son representaciones multidimensionales de los significados de las palabras.

# Accede al vector a través del atributo token.vector
print(doc[1].vector) # vector de la palabra "Arroz"

0.8463697235674686
[-0.33539    0.17083   -0.62339    2.017     -1.3763    -1.1457
 -0.72034    2.4666     2.508     -2.0996    -0.94729    0.31961
  2.4047     2.5066     0.7299     1.1566    -1.9779     2.9473
  0.74463    1.354      0.79521   -2.1274     1.7073    -1.0885
  1.5979    -1.3678     1.1434     5.4932     3.2241    -0.97814
  0.36803    0.80475    2.3822    -0.3006     0.97192    2.4423
  1.5567     2.0621     2.3309    -1.4673     1.673     -0.99992
 -0.93373    1.0229    -2.2532     0.49394   -1.2381    -0.53449
 -1.8662     3.048     -1.1243    -3.2422    -1.2709    -0.8209
  1.0236    -1.4516    -2.1556    -0.71552   -0.68864   -3.4261
 -0.38578    1.9612     0.57291   -1.9158     2.1337     0.29352
 -0.64589    0.94083   -0.63975    2.7674     0.72211    1.8376
  1.0876     2.6115     0.36064   -2.4631     1.1471     1.8923
  0.85355   -2.8397     0.99956    1.0787     0.29674    0.21393
  0.013852   1.4735    -1.8616     3.2102    -0.18652   -1.6293
 -1.6858     0.